# Data Explore

In [ ]:
import os
import ee
import geemap.core as geemap
import pandas as pd
import geopandas as gpd
import urllib.request

from IPython.display import Image

In [ ]:
from libs.ee import initialize_ee

ee = initialize_ee(get_ipython())

from libs.s2cloudless import S2Cloudless
from libs.sentinel import mask_s2_clouds, get_sentinel_image_thumbnail

# Load data

In [ ]:
directory = 'data/NZ_landslide_inventory/'
auckland_filepath = f'{directory}/cleaned_auckland.gpkg'
gdf = gpd.read_file(auckland_filepath, layer='landslides')
gdf = gdf[gdf['dateoccurence'].notnull()]
gdf.head()

In [ ]:
idx = 0
event = gdf.iloc[idx]
point = event.geometry.centroid
points = gdf.geometry.centroid
start_date = gdf['dateoccurence'].min().date().strftime("%Y-%m-%d")
end_date = ( gdf['dateoccurence'].min().date() + pd.Timedelta(days=91)).strftime("%Y-%m-%d")
point = ee.Geometry.Point(point.x, point.y)

In [ ]:
start_date, end_date

## Quick preview on map

In [ ]:
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(start_date, end_date)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

In [ ]:
# Convert GeoSeries to a list of ee.Geometry.Point
ee_points = [ee.Geometry.Point([p.x, p.y]) for p in points]

# Create an ee.FeatureCollection from the points
ee_features = [ee.Feature(p) for p in ee_points]
ee_feature_collection = ee.FeatureCollection(ee_features)

In [ ]:
# m = geemap.Map()
# m.set_center(point.coordinates().get(0).getInfo(), point.coordinates().get(1).getInfo(), 15)
# m.add_layer(dataset.min(), visualization, 'RGB')
# m.add_layer(point, {'color': 'red'}, 'My Pinpoint')
# # Add the FeatureCollection to the map
# m.addLayer(ee_feature_collection, {'color': 'red'}, 'Landslide Points')

# m

## Cloud removal

In [ ]:
s2cloudless = S2Cloudless(ee)
# point = points[0]
aoi = ee.Geometry.Point(point.coordinates().get(0).getInfo(), point.coordinates().get(1).getInfo()).buffer(1000)
# start_date
# end_date,
buffer=50
cloud_filter=60

In [ ]:
# s2_sr_cld_col_eval = s2cloudless._get_s2_sr_cld_col(aoi, start_date, end_date, cloud_filter)
# # As ever, a good idea to do a quick viusalization check
# Map = geemap.Map()
# Map.centerObject(aoi, 12)
# Map.addLayer(s2_sr_cld_col_eval, {'bands':['B4','B3','B2'], 'min':0, 'max':3000}, 'S2 collection (before)')
# Map.addLayer(point, {'color': 'red'}, 'My Pinpoint')
# Map

In [ ]:
# m_cloud = s2cloudless.cloud_layers_map(aoi, start_date, end_date, buffer=buffer, cloud_filter=cloud_filter)
# m_cloud.addLayer(point, {'color': 'red'}, 'My Pinpoint')
# m_cloud

In [ ]:
m_cloud_free = s2cloudless.cloud_free_map(aoi, start_date, end_date, buffer=buffer, cloud_filter=cloud_filter)
m_cloud_free.addLayer(point, {'color': 'red'}, 'My Pinpoint')
m_cloud_free

In [ ]:
out_dir = './data/2-data-explore-events/'
start_date_offset = 1
end_date_offset = 61 # s2 revisit time is 5 days, make sure we have images before the event

for idx, event in gdf.iterrows():
    filepath_rgb, filepath_with_marker = get_sentinel_image_thumbnail(
        ee,
        s2cloudless,
        event,
        idx,
        out_dir,
        start_date_offset=start_date_offset,
        end_date_offset=end_date_offset,
        buffer=buffer,
        cloud_filter=cloud_filter,
    )

    display(f"Filename {filepath_rgb}")
    display(Image(filename=filepath_with_marker))